In [ ]:
# https://github.com/dvschultz/ml-art-colabs/blob/master/Stylegan2_ada_Custom_Training.ipynb

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)]()


# <b> Custom Training StyleGan2-ADA </b>
StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

You're going to need at least 12 GB of RAM for the stylegan-2 to run. With Google Collab, you'd need pro or pro plus unfortunatley.

In [ ]:
!pip install numpy==1.19.5 # For some reason, need to downgrade to run.

     |████████████████████████████████| 14.8 MB 15.4 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 1.15.2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /tensorflow-1.15.2/python3.7
Requires: grpcio, google-pasta, tensorflow-estimator, keras-applications, numpy, absl-py, protobuf, opt-einsum, six, tensorboard, gast, wrapt, termcolor, wheel, keras-preprocessing, astor
Required-by: stable-baselines, magenta, kapre


In [ ]:
!nvidia-smi

Sun Mar 20 03:02:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# <b> Install Repo to Google Drive </b>
Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [ ]:
import os
if os.path.isdir("/content/drive/My Drive/colab-sg2-ada"):
    %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/My Drive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/My Drive/colab-sg2-ada/stylegan2-ada


In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

/content/drive/My Drive/colab-sg2-ada/stylegan2-ada


# <b> Convert dataset to .tfrecords </b>
<i> Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells. </i>

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [ ]:
# # #if you manually uploaded your dataset to Colab, unzip it
# zip_path = "/content/CAT1024.zip"

# !unzip {zip_path} -d /content/

Now that your image dataset is uploaded, we need to convert it to the .tfrecords format.

Depending on the resolution of your images and how many you have, this can take a while.

In [ ]:
#update this to the path to your image folder
dataset_path = "/content/drive/MyDrive/artists" #/content/drive/MyDrive/abstract abstract_tf for entire ~7500 images.

#give your dataset a name
dataset_name = "artists_tf" #abstract_tf for entire ~7500 images.

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

"""NOTE: I move the ./datasets/dataset_name/ to drive/My Drive so I don't have to rerun this cell."""

Loading images from "/content/drive/MyDrive/artists"
Creating dataset "./datasets/artists_tf"
dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 21309 images.


"NOTE: I move the ./datasets/dataset_name/ to drive/My Drive so I don't have to rerun this cell."

<b> Train a custom model </b>
We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [ ]:
!python train.py --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "artists_tf" # abstract_tf
#how often should the model generate samples and a .pkl file
snapshot_count = 5
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = True
"""Since mirrored vertically and horizontally, we now have x4 as many records."""

#metrics? 
metric_list = None
#augments
# augs = "bgcfnc"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
resume_from = "/content/drive/MyDrive/results_artists_gan/00002-artists_tf-mirror-mirrory-11gb-gpu-bgcfnc-resumecustom/network-snapshot-000060.pkl" # Location of latest .pkl weights. --resume={resume_from} 

!python train.py --outdir=/content/drive/MyDrive/results_artists_gan --snap={snapshot_count} --cfg=11gb-gpu --data=/content/drive/MyDrive/{dataset_name} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --augpipe=bgcfnc --resume={resume_from} 

tcmalloc: large alloc 4294967296 bytes == 0x55990819e000 @  0x7fec2ca90001 0x7fec29c9354f 0x7fec29ce3b58 0x7fec29ce7b17 0x7fec29d86203 0x559900000424 0x559900000120 0x559900074b80 0x55990006f66e 0x55990000236c 0x5599000437b9 0x5599000406d4 0x559900000c29 0x559900074e61 0x55990006f02f 0x5598fff40e2b 0x559900071633 0x55990006f02f 0x5598fff40e2b 0x559900071633 0x55990006f66e 0x5598fff40e2b 0x559900071633 0x5599000019da 0x55990006feae 0x55990006f02f 0x55990006ed43 0x559900139302 0x55990013967d 0x559900139526 0x5599001111d3
tcmalloc: large alloc 4294967296 bytes == 0x559a0819e000 @  0x7fec2ca8e1e7 0x7fec29c9346e 0x7fec29ce3c7b 0x7fec29ce435f 0x7fec29d86103 0x559900000424 0x559900000120 0x559900074b80 0x55990006f02f 0x559900001aba 0x559900070cd4 0x55990006f02f 0x559900001aba 0x559900070cd4 0x55990006f02f 0x559900001aba 0x559900070cd4 0x5599000019da 0x55990006feae 0x55990006f02f 0x559900001aba 0x5599000742c0 0x55990006f02f 0x559900001aba 0x559900070cd4 0x55990006f66e 0x55990000236c 0x55990004

While it’s training...
Once the above cell is running you should be training!

Don’t close this tab! Colab needs to be open and running in order to continue training. Every 15min 256x256x3 or 35min 1024x1024x3 on P100 (~45 min for T4) or so a new line should get added to your output, indicated its still training. Depending on you snapshot_count setting you should see the results folder in your Google drive folder fill with both samples (fakesXXXXXx.jpg) and model weights (network-snapshot-XXXXXX.pkl). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the resume_from path to continue training from the latest model.